In [ ]:
import pandas as pd
import numpy as np
import os

def procesar_archivo_bancario(nombre_archivo, categoria):
    # Diccionario para mapear las monedas según tus requerimientos
    mapa_monedas = {
        "Saldos en millones de pesos": "CLP",
        "Saldos en millones de dólares": "USD"
    }
    
    # Lista para almacenar los DataFrames de cada hoja
    data_consolidada = []
    
    # Cargar el archivo completo para obtener los nombres de las hojas
    excel_file = pd.ExcelFile(nombre_archivo)
    
    for hoja in excel_file.sheet_names:
        # 1. Leer metadatos (Subtipo y Moneda)
        # B3 es index 2, columna 1 (B). B4 es index 3.
        df_meta = pd.read_excel(nombre_archivo, sheet_name=hoja, header=None, nrows=5, usecols="B")
        subtipo = df_meta.iloc[2, 0]
        moneda_raw = df_meta.iloc[3, 0]
        moneda = mapa_monedas.get(moneda_raw, moneda_raw) # Si no coincide, deja el texto original
        
        # 2. Leer bloque de datos (B6:FK32)
        # header=0 indica que la fila 6 (index 5) son las columnas (Fechas)
        df_datos = pd.read_excel(
            nombre_archivo, 
            sheet_name=hoja, 
            skiprows=5,      # Empezamos en la fila 6
            nrows=27,        # De la fila 6 a la 32 hay 27 filas (excluyendo totales)
            usecols="B:FK"   # Rango de columnas B a FK
        )
        
        # Renombrar la columna B que contiene la institución
        df_datos = df_datos.rename(columns={df_datos.columns[0]: 'Institución Financiera'})
        
        # 3. Transformar de formato ancho a largo (Melt)
        # Esto convierte las columnas de fechas en una sola columna de 'Fecha'
        df_melted = df_datos.melt(
            id_vars=['Institución Financiera'], 
            var_name='Fecha', 
            value_name='Valor (millones)'
        )
        
        # 4. Agregar columnas de contexto
        df_melted['Tipo'] = categoria
        df_melted['Subtipo'] = subtipo
        df_melted['Tipo de moneda'] = moneda
        
        data_consolidada.append(df_melted)
        print(f"Procesada hoja {hoja} de {nombre_archivo}")

    return pd.concat(data_consolidada, ignore_index=True)

# --- EJECUCIÓN PRINCIPAL ---

archivos = [
    ('sdbcol_092021.xlsx', 'Colocación'),
    ('sdbdep_092021.xlsx', 'Captación'),
    ('sdbinv_092021.xlsx', 'Inversión')
]

dfs_finales = []

for archivo, tipo in archivos:
    if os.path.exists(archivo):
        df_temp = procesar_archivo_bancario(archivo, tipo)
        dfs_finales.append(df_temp)
    else:
        print(f"Advertencia: No se encontró el archivo {archivo}")

# Consolidar todo en un solo DataFrame
df_final = pd.concat(dfs_finales, ignore_index=True)

# Limpieza final de fechas (por si Excel las trae como strings o timestamps)
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')

# Reordenar columnas según tu solicitud
columnas_orden = ['Fecha', 'Institución Financiera', 'Tipo', 'Subtipo', 'Valor (millones)', 'Tipo de moneda']
df_final = df_final[columnas_orden]

# Mostrar resultado
df_final.head()